# **Tarda 4 minutos en corregir aprox**

# Desglose del enunciado

1. Tecnologías Clave:  
- **Python:** El lenguaje base.
- **gRPC:** Un protocolo de comunicación eficiente basado en Protocol Buffers (protobufs).
- **REST**: API tradicional basada en HTTP.
- **Redis**: Almacenamiento en memoria, probablemente usado para cacheo o mensajería.
- **Contenedores**: Seguramente usan Docker para encapsular cada microservicio.

2. Microservicios Involucrados
- **UserService**:   
-Maneja la gestión de usuarios, registro, login, recuperación de datos...    
-Probablemente almacene info en una base de datos.   

- **MessageService**:    
-Se encarga del envío y recepción de mensajes entre usuarios.       
-Puede interactuar con Redis para colas de mensajes o caché.       

- **NotificationService**
-Genera y envía notificaciones (push, emails, etc.).    
-Puede depender de eventos de otros servicios.    

- **FrontendService**:
-La interfaz que consume los demás servicios.    
-Posiblemente implemente REST para comunicarse con los microservicios.   

3. Estructura del repositorio:   

Hay un archivo **docker-compose.yml**, lo que indica que todos los servicios pueden levantarse juntos en contenedores.     

También está la carpeta proto_definitions/, que almacena las definiciones de Protocol Buffers (.proto), asegurando que **todos los servicios usen la misma estructura de datos**.   

***Cada microservicio tiene***    

📌 api_rest/app.py → Implementación en REST.   
📌 grpc/app.py → Implementación en gRPC.   
📌 grpc/proto_generated/ → Código generado a partir de los .proto.    
📌 requirements.txt → Dependencias necesarias.    
📌 Dockerfile → Define cómo se ejecuta el servicio en un contenedor.      


***Puntos Clave a Revisar***   

🔹 proto_definitions/

- Aquí están los .proto que definen los mensajes y servicios.
- Si hay problemas de comunicación, revisar si los servicios usan la misma versión.


🔹 docker-compose.yml   

- Contiene la configuración para levantar los microservicios juntos.
- Posibles errores:   
Problemas de red entre contenedores.    
Dependencias que no se inician en el orden correcto.    

🔹 proto_generated/

- Código generado a partir de los .proto.
- Si hay errores en la comunicación gRPC, puede ser necesario regenerar los archivos.


&nbsp;

# Puertos de los servicios

Cada servicio tiene dos puertos: 
- 1️⃣ HTTP: Usado para REST API.
- 2️⃣ gRPC: Usado para la comunicación interna eficiente.

🔹 FrontendService solo usa gRPC (3030), lo que significa que toda la comunicación con el frontend debe hacerse mediante gRPC y no REST.    
🔹 Redis está corriendo en el puerto 6379, lo que indica que puede estar siendo usado para almacenamiento en caché o para un sistema de Pub/Sub.    

* ⚠️ Posibles Problemas Relacionados con los Puertos    

✅ Servicios no conectándose correctamente     

-Revisar si en el código están llamando a los puertos correctos al hacer peticiones gRPC o REST.    
-MessageService debe estar llamando a UserService en 8080 (REST) o 9797 (gRPC).   
-NotificationService debe comunicarse con FrontendService en 3030 (gRPC).   


✅ Conflictos de puertos

-Si algún servicio ya está usando el puerto asignado, puede haber fallos de conexión.     
-Verificar con docker ps y netstat -tulnp (en Linux).        


✅ Redis no accesible         
    
-Si las notificaciones se guardan en Redis y este no está corriendo o el puerto es incorrecto, NotificationService puede fallar.    
-Se puede probar conexión con redis-cli -h redis -p 6397 ping.       

&nbsp;

# Tareas🎯🎯🎯🎯

### 🛠 Tarea 1: Arreglar la comunicación entre MessageService y UserService

📌 **Problema**: Hay errores en la comunicación interna entre estos servicios, lo que interrumpe su funcionalidad.       
  
* Falta o inconsistencia en las definiciones de user.proto o message.proto.
* MessageService no está llamando correctamente a UserService.
* Problemas con la configuración de gRPC o REST entre estos servicios.
* Docker no está exponiendo correctamente los puertos o los contenedores no pueden comunicarse.
 
📌 **Posible solución**:    

1️⃣ Revisar message.proto y user.proto en proto_definitions/ para ver si están bien definidos.   

2️⃣ Verificar la generación de código en proto_generated/, asegurándose de que ambos servicios usan la misma versión.    

3️⃣ Inspeccionar los logs (docker logs <container_id>) para ver errores específicos.    

4️⃣ Hacer pruebas directas de comunicación entre ambos servicios.    


&nbsp;

### 📩 Tarea 2: Implementar la comunicación de notificación al frontend

📌 **Objetivo**: Cuando se cree un mensaje en MessageService, se debe notificar al usuario en FrontendService.    

📌 **Pasos a implementar**:    

- 1️⃣ 🚀 Desencadenador: 

-Cuando un usuario recibe un mensaje, MessageService debe enviar un evento a NotificationService.    
-Este evento se puede manejar con una llamada gRPC o Redis (pub/sub).    


- 2️⃣ 📢 Creación de la notificación:

-NotificationService verifica si el usuario destinatario está suscrito a las notificaciones.    
-Todos los usuarios están suscritos por defecto, pero puede haber una opción para desuscribirse.  


- 3️⃣ 📡 Enviar la notificación al FrontendService:    

-NotificationService usa gRPC para enviar la notificación a FrontendService.    
-FrontendService recibe la notificación y la muestra en tiempo real.    


- 4️⃣ 💾 Persistencia en Redis:    

-La notificación se almacena temporalmente en Redis, probablemente como una lista de notificaciones pendientes.     
-Esto permite recuperar las notificaciones si el usuario estaba desconectado.    

&nbsp;

# Guías

**User service:**
1. Apirest:
- El servicio no requiere autenticación, lo cual es curioso, especialmente para modificar/eliminar usuarios.
- El email es clave única para cada usuario, usado en las actualizaciones (PUT) y eliminaciones (DELETE).
- Los datos de los usuarios se almacenan en Redis (DB: 0) en formato JSON.

2. gRPC:
- AuthenticateUser: Confirma si el usuario y la contraseña son correctos.
- CheckUserExists: Permite verificar si un usuario está registrado.
- ListUsers: Devuelve una lista de todos los usuarios en la base de datos.

3. Redis (DB 0):
- Redis debe estar corriendo en el puerto 6397 y debe ser accesible.
- Si los usuarios se almacenan en Redis y este se cae, se pierden los datos (a menos que haya persistencia).
- password debería estar hasheada para seguridad.

```bash
#Ejemplo de Usuario
{
  "name": "User's Name",
  "email": "user@example.com",
  "password": "hashed_password"
}

&nbsp;

**Message Service:**
1. Apirest:
- Solo tiene un endpoint: /list_conversations (GET).
- No requiere autenticación, lo que podría ser un problema de seguridad.
- Permite listar conversaciones de un usuario dado.

2. gRPC:
-SendMessage: Envía un mensaje de un usuario a otro.    
-GetMessages: Recupera todos los mensajes de un usuario.    
-No menciona autenticación, lo que puede permitir acceso no autorizado.    

3. Redis (DB: 1):

* Conversaciones:    
-Clave: conversation:{user_email_1}:{user_email_2} (Hash).    
Almacena metadatos de conversaciones entre pares de usuarios.  

* Mensajes individuales:    
-Clave: message:{message_id} (Cadena JSON).    
-Contiene información del mensaje (remitente, receptor, contenido, timestamp).    

* Contador de mensajes:    
-Clave: message_id_counter, para IDs únicos.    

* Lista de conversaciones por usuario:   
-Clave: user:{user_email}:conversations (Set).    
-Guarda todas las conversaciones en las que participa el usuario.   

***Posibles problemas:***  
✅ Redis debe estar en el puerto 6397 y operativo.    
✅ Si message_id_counter no se incrementa, los mensajes pueden sobrescribirse.   
✅ Falta autenticación para evitar acceso indebido a mensajes.   
✅ No hay endpoint para marcar mensajes como leídos o eliminarlos.   
 
```bash

#Ejemplo de mensaje almacenado en Redis
{
  "id": 1,
  "sender_email": "sender@example.com",
  "receiver_email": "receiver@example.com",
  "content": "Hello, how are you?",
  "timestamp": "2025-01-14T12:00:00Z"
}


&nbsp;

***Notification Service:***

1. Apirest:
- Solo tiene un endpoint: /list_notifications (GET).
- No requiere autenticación, lo que puede permitir acceso no autorizado a notificaciones.
- Permite listar notificaciones de un usuario dado.

2. gRPC:

- CreateNotification: Crea una notificación para un usuario receptor. Debe incluir un parámetro "read": bool.
- CheckUserSubscribed: Verifica si un usuario está suscrito a las notificaciones.
- SubscribeUser: Permite que un usuario se suscriba a las notificaciones.
- UnsubscribeUser: Permite que un usuario cancele su suscripción.

3. Redis (DB 2):

* Notificaciones:    
-Clave: notifications:{user_email} (Lista).   
-Almacena notificaciones recibidas por el usuario como objetos JSON.    

* Suscripciones:     
-Clave: subscriptions (Hash).   
-Almacena el estado de suscripción de los usuarios (email → suscrito o no).    


***Posibles problemas:***    
✅ Redis debe estar en el puerto 6397 y operativo.     
✅ Falta autenticación en la API REST, lo que podría permitir que cualquier usuario acceda a las notificaciones de otro.    
✅ No hay un mecanismo para marcar notificaciones como leídas a través de la API.   
✅ No se menciona si las notificaciones antiguas se eliminan o si Redis podría llenarse con el tiempo.    

```bash
#Ejemplo de notificación almacenada en Redis
{
  "sender_email": "bob@example.com",
  "receiver_email": "asdf@example.com",
  "timestamp": "2025-01-13T19:31:14.207231",
  "read": false
}

#Ejemplo de suscripción en Redis
{
  "user_email": "notification_sender@example.com",
  "subscribed": true
}

&nbsp;

***Frontend Service:***

1. gRPC:
- ReceiveNotification: Recibe notificaciones enviadas desde NotificationService.
- No tiene endpoints HTTP, solo gRPC.

2. Funcionalidad:
- Actúa como el punto de entrega final de las notificaciones.
- No almacena datos, solo recibe las notificaciones en tiempo real.

***Posibles problemas:***    
✅ No hay autenticación, lo que podría permitir que cualquier cliente reciba notificaciones.   
✅ No se menciona cómo las notificaciones se muestran o manejan en el frontend.    
✅ Si FrontendService está caído, se podrían perder notificaciones si no hay reintentos de envío.   

&nbsp;

&nbsp;

# Resumen Final del Enunciado:

1. El proyecto está dockerizado y debe ejecutarse usando Docker y Docker Compose.
2. El archivo docker-compose.yml no debe modificarse.
3. Cada microservicio tiene su propio Dockerfile que se puede modificar si es necesario (aunque no es obligatorio para resolver el desafío).

* Pasos para Correr el Proyecto:

-Verificar instalación de Docker y docker compose.        

-Construir las imágenes Docker.       

-Iniciar los contenedores en modo independiente.

- Pruebas:
  
-En el directorio tests/ hay ejemplos de casos de prueba.      
-Se recomienda implementar pruebas adicionales para cubrir más escenarios.      



# REDIS


In [6]:
from flask import Flask, jsonify
import redis
import os

# Crear la aplicación Flask
app = Flask(__name__)

# Función de conexión a Redis
def check_redis_connection():
    try:
        redis_host = os.getenv("REDIS_HOST", "localhost")
        redis_port = os.getenv("REDIS_PORT", 6379)

        r = redis.StrictRedis(host=redis_host, port=redis_port, db=0, socket_connect_timeout=5)
        
        # Intentar hacer un ping para verificar la conexión
        r.ping()
        return {"message": "Conexión exitosa a Redis"}
    except redis.ConnectionError as e:
        return {"error": f"Error de conexión a Redis: {e}"}

# Crear contexto de aplicación y probar la conexión a Redis
with app.app_context():
    result = check_redis_connection()
    print(result)


{'message': 'Conexión exitosa a Redis'}


# Pruebas API